# [Conduct a Vector Similarity Search](https://milvus.io/docs/v2.3.0-beta/search.md)

> This topic describes how to search entities with Milvus.

A vector similarity search in Milvus calculates the distance between query vector(s) and vectors in the collection with specified similarity metrics, and returns the most similar results. By specifying a boolean expression that filters the scalar field or the primary key field, you can perform a hybrid search or even a search with Time Travel.

# Connect to milvus

In [1]:
#r "nuget: IO.Milvus, 2.2.2-preview.1"
#!import config/Settings.cs

using Milvus.Client;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

//Connect to milvus
(string endpoint, int port, string userName, string password) = Settings.LoadFromFile();

MilvusClient milvusClient = default;

milvusClient = new MilvusClient(endpoint, port, userName, password, null);
milvusClient

Installed Packages IO.Milvus, 2.2.2-preview.1

Address,localhost:19530


# Load collection

All search and query operations within Milvus are executed in memory. Load the collection to memory before conducting a vector similarity search.

In [14]:
string collectionName = "book";
MilvusCollection collection = milvusClient.GetCollection(collectionName);

if(!(await milvusClient.HasCollectionAsync(collectionName)))
{
    Console.WriteLine($"Collection {collectionName} not exist");
    return;
}

await collection.LoadAsync();

//Waiting for collection loaded
collection.WaitForCollectionLoadAsync(timeout: TimeSpan.FromSeconds(10));

MilvusCollectionDescription collectionInfo = await collection.DescribeAsync();
collectionInfo

Milvus.Client.MilvusCollectionDescription Aliases [ ] CollectionName book CollectionId 443665421144165275 ConsistencyLevel Session CreationTimestamp 1692449879719 Schema Milvus.Client.CollectionSchema Name book Description Fields index value 0 Milvus.Client.FieldSchema Name book_id DataType Int64 IsPrimaryKey True AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension <null> State FieldCreated FieldId 100 1 Milvus.Client.FieldSchema Name word_count DataType Int64 IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension <null> State FieldCreated FieldId 101 2 Milvus.Client.FieldSchema Name book_name DataType VarChar IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength 256 Dimension <null> State FieldCreated FieldId 102 3 Milvus.Client.FieldSchema Name book_intro DataType FloatVector IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension 2 State FieldCreated FieldId 103 EnableDynamicFields False ShardsNum 1 StartPositions (empty)

# Prepare search parameters

| Parameter | Description | Options |
| --- | --- | --- |
| TopK | Number of the most similar results to return. | N/A |
| Params | Search parameter(s) specific to the index. | See [Vector Index](https://milvus.io/docs/v2.3.0-beta/index.md) for more information.

# Conduct a vector search

🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍

In [9]:
List<string> search_output_fields = new() { "book_id" };
List<List<float>> search_vectors = new() { new() { 0.1f, 0.2f } };
SearchResults searchResult = await collection.SearchAsync(
    "book_intro",
    new ReadOnlyMemory<float>[] { new[] { 0.1f, 0.2f } },
    SimilarityMetricType.L2,
    limit: 2);

searchResult

Milvus.Client.SearchResults CollectionName book FieldsData (empty) Ids Milvus.Client.MilvusIds LongIds [ 710, 1588 ] StringIds <null> NumQueries 1 Scores [ 2.7538933E+15, 3.5035509E+15 ] Limit 2 Limits [ 2 ]

| Parameter         | Description                                                  | Options                                                      |
| ----------------- | ------------------------------------------------------------ | ------------------------------------------------------------ |
| CollectionName    | Name of the collection to load.                              | N/A                                                          |
| MetricType        | Metric type used for search.                                 | This parameter must be set identical to the metric type used for index building. |
| OutFields         | Name of the field to return.                                 | Vector field is not supported in current release.            |
| Vectors           | Vectors to search with.                                      | N/A                                                          |
| VectorFieldName   | Name of the field to search on.                               | N/A                                                          |
| Expr              | Boolean expression used to filter attribute.                  | See Boolean Expression Rules for more information.           |
| ConsistencyLevel  | The consistency level used in the query.                      | STRONG, BOUNDED, and EVENTUALLY.                             |

# Release the Collection.

🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃

> Release the collection loaded in Milvus to reduce memory consumption when the search is completed.

In [13]:
await collection.ReleaseAsync();

MilvusCollectionDescription collectionInfo = await collection.DescribeAsync();
collectionInfo

Milvus.Client.MilvusCollectionDescription Aliases [ ] CollectionName book CollectionId 443665421144165275 ConsistencyLevel Session CreationTimestamp 1692449879719 Schema Milvus.Client.CollectionSchema Name book Description Fields index value 0 Milvus.Client.FieldSchema Name book_id DataType Int64 IsPrimaryKey True AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension <null> State FieldCreated FieldId 100 1 Milvus.Client.FieldSchema Name word_count DataType Int64 IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension <null> State FieldCreated FieldId 101 2 Milvus.Client.FieldSchema Name book_name DataType VarChar IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength 256 Dimension <null> State FieldCreated FieldId 102 3 Milvus.Client.FieldSchema Name book_intro DataType FloatVector IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension 2 State FieldCreated FieldId 103 EnableDynamicFields False ShardsNum 1 StartPositions (empty)